# Run Simulations

## Setup

Importage

In [1]:
# Generic stuff

import warnings
warnings.filterwarnings('ignore')

import numpy as np,pandas as pd
from numpy import exp,sin,cos,sqrt
from scipy.signal import welch,periodogram
from datetime import datetime
from itertools import product
from copy import copy
from joblib import Parallel,delayed
from mne.connectivity import spectral_connectivity

# Vizualization stuff

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import Image

Simulation function

In [2]:
# Notes:
# - time series not returned by default (to avoid memory errors). Will be returned 
#   if return_ts=True
# - freq vs. amp param sweep has default range within which max freqs are calculated 
#   of 5-95 Hz

def run_sim(wee = 0.5,wei = 1.,wie = -2.,wii = -0.5,wertn = 0.6,weth = .6,
            wthi = 0.2,wthe = 1.65,wrtnth = -2.,wthrtn = 2.,D_e = .0001,D_i= .0001,
            D_th = 0.0001,D_rtn = 0.0001,T = 1024*4,P=1,Q = 1,K = 1,Dt = 0.001,
            dt = 0.1,gain = 20.,threshold = 0.,Pi = 3.14159,g = -0.9,a_e = 0.3,
            a_i = 0.5,a_th = 0.2,a_rtn = 0.2,i_e = -0.35,i_i = -0.3,i_th = 0.5,
            i_rtn = -0.8,tau1  = 20.,tau2 = 5.,I_o = 0,T_transient=1000,
            stim_freq=35.,stim_amp=0.5,return_ts=False,compute_connectivity=False):
    
    n_nodes = K
    
    x1 = n_nodes/2.
    x2 = x1+20.

    # Neuronal response function
    def f(u,gain=gain,threshold=threshold):
        output= 1./(1.+exp(-gain*(u-threshold))); 
        return output 
    
    # Initialize variables
    Xi_e = np.random.uniform(low=-1.,high=1.,size=[K,T+T_transient])
    Xi_i = np.random.uniform(low=-1.,high=1.,size=[K,T+T_transient])
    Xi_th = np.random.uniform(low=-1.,high=1.,size=[K,T+T_transient])
    Xi_rtn = np.random.uniform(low=-1.,high=1.,size=[K,T+T_transient])
    
    e = np.zeros_like(Xi_e) 
    i = np.zeros_like(Xi_i)
    th = np.zeros_like(Xi_th)
    rtn = np.zeros_like(Xi_rtn)
    
    Ts = np.arange(0,T+T_transient)
    

    #state_input = ((Ts>x1+1)&(Ts<x2)).astype(float)
    state_input = 1. # not sure what this is suppose do be doing...
    
    stim = state_input * stim_amp*sin(2*Pi*stim_freq*Ts*Dt)

    # If I do this, it changes the time series dramatically (halves the period...)
    #e = Xi_e
    #i = Xi_i
    #th = Xi_th
    #rtn = Xi_rtn
    # ...so initialize just first state with random variables instead...
    e[:,0] = np.random.randn(n_nodes)
    i[:,0] = np.random.randn(n_nodes)
    th[:,0] = np.random.randn(n_nodes)
    rtn[:,0] = np.random.randn(n_nodes)
    
    # Integration loop
    
    for t in Ts[:-1]:
        
        Cortico_cortical_input = f(e[:,t]).mean()
        
        # Cortico-cortical connections
        weefe = wee*f(e[:,t])
        wiifi = wii*f(i[:,t])
        wiefi = wie*f(i[:,t])
        weife = wei*f(e[:,t]) 
        gccin = g*Cortico_cortical_input
            
        # Cortico-thalamic connections
        wethfe = weth*f(e[:,t-tau1]) 
        wertnfe = wertn*f(e[:,t-tau1])
            
        # Thalamo-cortical connections
        if t<=tau1:
            wthefth=wthifth=0
        else:
            wthefth = wthe*f(th[:,t-tau1])
            wthifth = wthi*f(th[:,t-tau1])
            
        # Thalamo-thalamic connections
        if t<=tau2:
            wthrtnfth=wrtnthfrtn=0
        else:
            wrtnthfrtn = wrtnth*f(rtn[:,t-tau2])
            wthrtnfth = wthrtn*f(th[:,t-tau2])

        # Noise inputs
        e_noise = sqrt(2.*D_e*dt)*Xi_e[:,t]
        i_noise = sqrt(2.*D_i*dt)*Xi_i[:,t]
        th_noise = sqrt(2.*D_th*dt)*Xi_th[:,t]
        rtn_noise = sqrt(2.*D_rtn*dt)*Xi_rtn[:,t]
            
            
        # Collate inputs
        e_inputs = weefe + wiefi + wthefth + gccin + i_e + stim[t]
        i_inputs = weife + wiifi + wthifth + i_i
        th_inputs = wethfe + wrtnthfrtn + I_o + i_th
        rtn_inputs = wthrtnfth + wertnfe+ i_rtn
          
        # d/dt
        e[:,t+1]   = e[:,t]   + dt*a_e*(-e[:,t]     + e_inputs)   + e_noise
        i[:,t+1]   = i[:,t]   + dt*a_i*(-i[:,t]     + i_inputs)   + i_noise
        th[:,t+1]  = th[:,t]  + dt*a_th*(-th[:,t]   + th_inputs)  + th_noise
        rtn[:,t+1] = rtn[:,t] + dt*a_rtn*(-rtn[:,t] + rtn_inputs) + rtn_noise

            

    # Concatenate sim time series
    df_all = pd.concat({'e': pd.DataFrame(e.T[T_transient:]),
                        'i': pd.DataFrame(i.T[T_transient:]),
                        'th':pd.DataFrame(th.T[T_transient:]),
                        'rtn': pd.DataFrame(rtn.T[T_transient:]),
                        'stim': pd.DataFrame(stim.T[T_transient:])},axis=1)
    df_all.index = (Ts[T_transient:]-T_transient)*(Dt*1000.)

    # Compute power spectra:
    
    # 1. Welch's method
    tmp = welch(df_all.T,fs= 1./Dt) 
    df_wel = pd.DataFrame(tmp[1].T,index=tmp[0])
    df_wel.columns = df_all.columns
    
    # 2. Periodogram
    tmp = periodogram(df_all.T,fs= 1./Dt) 
    df_per = pd.DataFrame(tmp[1].T,index=tmp[0])
    df_per.columns = df_all.columns
    

    if compute_connectivity:
        
        cols = df_all.columns
        data = df_all.values.copy().T[np.newaxis,:,:]
        
        spec_conn_coh = spectral_connectivity(data=data,method='coh',sfreq=1./Dt,verbose=False)
        con_coh,freqs_coh,times_coh,n_epochs_coh,n_tapers_coh = spec_conn_coh
        df_con_coh = pd.concat({f: pd.DataFrame(con_coh[:,:,f_it].T,columns=
                                                cols,index=cols) \
                    for f_it,f in enumerate(freqs_coh)})
        df_con_coh.index.names = ['freq', 'pop', 'region']

        spec_conn_plv = spectral_connectivity(data=data,method='plv',sfreq=1./Dt,verbose=False)
        con_plv,freqs_plv,times_plv,n_epochs_plv,n_tapers_plv = spec_conn_plv
        df_con_plv = pd.concat({f: pd.DataFrame(con_plv[:,:,f_it].T,columns=
                                                cols,index=cols) \
                    for f_it,f in enumerate(freqs_plv)})
        df_con_plv.index.names = ['freq', 'pop', 'region']
        
    else:
        df_con_coh=df_con_plv=None
        
    # this is necessary to stop memory error crashes
    if return_ts==False: df_all=None
    
    return df_all,df_wel,df_per,df_con_coh,df_con_plv

## Single exemplary parameter set sim runs

No stim

In [3]:
sim_rest_nostim = run_sim(I_o=0.,K=100,stim_amp=0.,return_ts=True)
sim_rest_nostim_ts,_,sim_rest_nostim_psd,_,_ = sim_rest_nostim

sim_task_nostim = run_sim(I_o=1.5,K=100,stim_amp=0.,return_ts=True)
sim_task_nostim_ts,_,sim_task_nostim_psd,_,_ = sim_task_nostim

Stim 27 Hz

In [4]:
sim_rest_stim27Hz = run_sim(I_o=0.,K=100,stim_amp=0.5,stim_freq=27.,return_ts=True)
sim_rest_stim27Hz_ts,_,sim_rest_stim27Hz_psd,_,_ = sim_rest_stim27Hz

sim_task_stim27Hz = run_sim(I_o=1.5,K=100,stim_amp=0.5,stim_freq=27.,return_ts=True)
sim_task_stim27Hz_ts,_,sim_task_stim27Hz_psd,_,_ = sim_task_stim27Hz

Save to file

In [5]:
sim_rest_nostim_ts.to_hdf('sim_singlepsets.hdf', 'sim_rest_nostim_ts')
sim_task_nostim_ts.to_hdf('sim_singlepsets.hdf', 'sim_task_nostim_ts')
sim_rest_nostim_psd.to_hdf('sim_singlepsets.hdf', 'sim_rest_nostim_psd')
sim_task_nostim_psd.to_hdf('sim_singlepsets.hdf', 'sim_task_nostim_psd')

sim_rest_stim27Hz_ts.to_hdf('sim_singlepsets.hdf', 'sim_rest_stim27Hz_ts')
sim_task_stim27Hz_ts.to_hdf('sim_singlepsets.hdf', 'sim_task_stim27Hz_ts')
sim_rest_stim27Hz_psd.to_hdf('sim_singlepsets.hdf', 'sim_rest_stim27Hz_psd')
sim_task_stim27Hz_psd.to_hdf('sim_singlepsets.hdf', 'sim_task_stim27Hz_psd')

## Stim amplitude vs. Stim frequency parameter space sim runs

Parameter sweep and parallelization wrappers

In [3]:
def pse_par(paramslist,sim_func=run_sim,n_jobs=2,temp_folder = '/tmp'):
    
    # Run parallel sims
    sim_res = Parallel(n_jobs=n_jobs,temp_folder=temp_folder)\
             (delayed(sim_func)(**p)\
             for p in paramslist)

    return sim_res       


def FreqVsAmpPsweep(fixed_params,freqs,amps,n_jobs=2,verbose=True,
                    temp_folder='/tmp',outfile=None,outpfx=None,
                    concat_res=False,return_ts=False,peak_freq_range=[5,95],
                    compute_connectivity=False):
    
    pfr = peak_freq_range
    
    if verbose == True:
        start = datetime.now()
        print 'started: %s' %start
        
    # Set up params list
    combs = list(product(freqs,amps))
    paramslist = []
    for f,a in combs:
        newparams = copy(fixed_params)
        newparams['stim_freq'] = f
        newparams['stim_amp'] = a
        newparams['return_ts'] = return_ts
        newparams['compute_connectivity'] = compute_connectivity
        paramslist.append(newparams)

    # Run parallel sims
    sim_res = pse_par(paramslist,sim_func = run_sim,n_jobs=n_jobs,
                      temp_folder=temp_folder)
    
    # Assemble outputs
    _all_dat,_all_wel,_all_per,_all_coh,_all_plv = {},{},{},{},{}
    for (f,a),(dat,wel,per,coh,plv) in zip(combs,sim_res):
        _all_dat[f,a] = dat
        _all_wel[f,a] = wel
        _all_per[f,a] = per
        _all_coh[f,a] = coh
        _all_plv[f,a] = plv

    # added in this option because getting memory error if the param space is too big 
    if concat_res == False:
        res_dict = dict(_all_dat=_all_dat,_all_per=_all_per,_all_wel=_all_wel,
                        _all_coh=_all_coh,_all_plv=_all_plv,paramslist=paramslist)
    else:
        
        if return_ts == True:
            all_dat = pd.concat(_all_dat)
            all_dat.index.names = ['stim_freq', 'stim_amp', 't']
            all_dat.columns.names = ['pop', 'region']
        else:
            all_dat = None
            
        all_wel = pd.concat(_all_wel)
        all_wel.index.names = ['stim_freq', 'stim_amp','Hz']
        all_wel.columns.names = ['pop', 'region']

        all_per = pd.concat(_all_per)
        all_per.index.names = ['stim_freq','stim_amp', 'Hz']
        all_per.columns.names = ['pop', 'region']
        
        all_wel_maxfreqs = all_wel.unstack(['stim_freq','stim_amp']).loc[pfr[0]:pfr[1]]\
                                  .idxmax().unstack('region').mean(axis=1).unstack('pop')
        
        all_wel_maxamps = all_wel.unstack(['stim_freq', 'stim_amp']).loc[pfr[0]:pfr[1]]\
                                 .unstack('region').mean(axis=1).unstack('pop')
    
        all_per_maxfreqs = all_per.unstack(['stim_freq','stim_amp']).loc[pfr[0]:pfr[1]]\
                               .idxmax().unstack('region').mean(axis=1).unstack('pop')
        
        all_per_maxamps = all_per.unstack(['stim_freq', 'stim_amp']).loc[pfr[0]:pfr[1]]\
                              .max().unstack('region').mean(axis=1).unstack('pop')

        res_dict = dict(all_dat=all_dat,all_per=all_per,all_wel=all_wel,
                        all_wel_maxfreqs=all_wel_maxfreqs,
                        all_wel_maxamps=all_wel_maxamps,
                        all_per_maxfreqs=all_per_maxfreqs,
                        all_per_maxamps=all_per_maxamps)

    
    if verbose == True:
        
        finish = datetime.now()
        dur = str(finish - start)
        print 'finished: %s' %finish
        print 'duration: %s' %dur
        
        
    
    return res_dict

Run sims

In [ ]:
freqs = np.arange(0,100,1)
amps = np.arange(0.,2.05,0.05)
fixed_params = {'I_o': 0., 'K':10}
sim_rest_psweep = FreqVsAmpPsweep(fixed_params,freqs,amps,n_jobs=4,concat_res=False)

started: 2018-03-16 07:11:04.084186


In [ ]:
tmp = pd.concat(sim_rest_psweep['_all_per'])['e'].mean(axis=1)
tmp.index.names = ['stim_freq','stim_amp', 'Hz']
tmp = tmp.unstack(['stim_freq', 'stim_amp']).idxmax().unstack('stim_freq')
sim_rest_psweep_maxfreqs = tmp

tmp = pd.concat(sim_rest_psweep['_all_per'])['e'].mean(axis=1)
tmp.index.names = ['stim_freq','stim_amp', 'Hz']
tmp = tmp.unstack(['stim_freq', 'stim_amp']).max().unstack('stim_freq')
sim_rest_psweep_maxamps = tmp

sim_rest_psweep_maxfreqs.to_hdf('sim_psweeps.hdf', 'sim_rest_psweep_maxfreqs')
sim_rest_psweep_maxamps.to_hdf('sim_psweeps.hdf', 'sim_rest_psweep_maxamps')

In [ ]:
del sim_rest_psweep

In [ ]:
freqs = np.arange(0,100,1)
amps = np.arange(0.,1.01,0.01)
fixed_params = {'I_o': 1.5, 'K':10}
sim_task_psweep = FreqVsAmpPsweep(fixed_params,freqs,amps,n_jobs=4,concat_res=False)

In [ ]:
tmp = pd.concat(sim_task_psweep['_all_per'])['e'].mean(axis=1)
tmp.index.names = ['stim_freq','stim_amp', 'Hz']
tmp = tmp.unstack(['stim_freq', 'stim_amp']).idxmax().unstack('stim_freq')
sim_task_psweep_maxfreqs = tmp

tmp = pd.concat(sim_task_psweep['_all_per'])['e'].mean(axis=1)
tmp.index.names = ['stim_freq','stim_amp', 'Hz']
tmp = tmp.unstack(['stim_freq', 'stim_amp']).max().unstack('stim_freq')
sim_task_psweep_maxamps = tmp

sim_task_psweep_maxfreqs.to_hdf('sim_psweeps.hdf', 'sim_task_psweep_maxfreqs')
sim_task_psweep_maxamps.to_hdf('sim_psweeps.hdf', 'sim_task_psweep_maxamps')